In [1]:
import pyjuice as juice
import torch
import torchvision
import time
from torch.utils.data import TensorDataset, DataLoader
import pyjuice.nodes.distributions as dists
import numpy as np
import pandas as pd

import seaborn as sns
import pandas as pd
import numpy as np
import importlib
import os
import sys
import matplotlib

In [2]:
# train_dataset = torchvision.datasets.MNIST(root = "./data", train = True, download = True)
# valid_dataset = torchvision.datasets.MNIST(root = "./data", train = False, download = True)

# train_data = train_dataset.data.reshape(60000, 28*28)
# valid_data = valid_dataset.data.reshape(10000, 28*28)

data_dir = "genetic_pc/data1kg"
file_name = "805.data"

def load(file_name, data_dir):
    filename = f"{data_dir}/{file_name}"
    dataframe = pd.read_csv(filename, header=None,
                            true_values=["1", "2", "3"],
                            false_values=["0"], dtype=object)
    data = dataframe.iloc[:, 0].str.split(' ')
    return np.array([np.array(entry) for entry in data])

data = load(file_name, data_dir)
data = data.astype(np.int8)
print(data.shape)

total_size = len(data)
train_size = int(0.8 * total_size)
valid_size = total_size - train_size

indices = np.random.permutation(total_size)
train_indices, valid_indices = indices[:train_size], indices[train_size:]
train_data, valid_data = data[train_indices], data[valid_indices]

train_data = torch.tensor(train_data, dtype=torch.uint8)
valid_data = torch.tensor(valid_data, dtype=torch.uint8)

print(train_data.shape)
print(valid_data.shape)

train_loader = DataLoader(
    dataset = TensorDataset(train_data),
    batch_size = 512,
    shuffle = True,
    drop_last = True
)
valid_loader = DataLoader(
    dataset = TensorDataset(valid_data),
    batch_size = 512,
    shuffle = False,
    drop_last = True
)

(5008, 805)
torch.Size([4006, 805])
torch.Size([1002, 805])


In [3]:
device = torch.device("cuda:0")

# The data is required to construct the backbone Chow-Liu Tree structure for the HCLT
ns = juice.structures.HCLT(
    train_data.float().to(device),
    num_latents = 128
)

pc = juice.compile(ns)
pc.to(device)

optimizer = juice.optim.CircuitOptimizer(pc, lr = 0.1, pseudocount = 0.1, method = "EM")
scheduler = juice.optim.CircuitScheduler(
    optimizer,
    method = "multi_linear",
    lrs = [0.9, 0.1, 0.05],
    milestone_steps = [0, len(train_loader) * 100, len(train_loader) * 350]
)

Compiling 8 TensorCircuit layers...


100%|██████████| 8/8 [00:01<00:00,  5.72it/s]


In [4]:
for batch in train_loader:
    x = batch[0].to(device)

    lls = pc(x, record_cudagraph = True)
    lls.mean().backward()
    break

In [5]:
for epoch in range(1, 350+1):
    t0 = time.time()
    train_ll = 0.0
    for batch in train_loader:
        x = batch[0].to(device)

        # Similar to PyTorch optimizers zeroling out the gradients, we zero out the parameter flows
        optimizer.zero_grad()

        # Forward pass
        lls = pc(x)

        # Backward pass
        lls.mean().backward()

        train_ll += lls.mean().detach().cpu().numpy().item()

        # Perform a mini-batch EM step
        optimizer.step()
        scheduler.step()

    train_ll /= len(train_loader)

    t1 = time.time()
    test_ll = 0.0
    for batch in valid_loader:
        x = batch[0].to(pc.device)
        lls = pc(x)
        test_ll += lls.mean().detach().cpu().numpy().item()

    test_ll /= len(valid_loader)
    t2 = time.time()

    print(f"[Epoch {epoch}/{350}][train LL: {train_ll:.2f}; val LL: {test_ll:.2f}].....[train forward+backward+step {t1-t0:.2f}; val forward {t2-t1:.2f}] ")

[Epoch 1/350][train LL: -1292.32; val LL: -404.04].....[train forward+backward+step 0.77; val forward 0.00] 
[Epoch 2/350][train LL: -402.48; val LL: -401.70].....[train forward+backward+step 0.23; val forward 0.00] 
[Epoch 3/350][train LL: -400.74; val LL: -400.74].....[train forward+backward+step 0.23; val forward 0.00] 
[Epoch 4/350][train LL: -399.57; val LL: -400.04].....[train forward+backward+step 0.23; val forward 0.00] 
[Epoch 5/350][train LL: -398.94; val LL: -399.46].....[train forward+backward+step 0.23; val forward 0.00] 
[Epoch 6/350][train LL: -398.36; val LL: -398.93].....[train forward+backward+step 0.23; val forward 0.00] 
[Epoch 7/350][train LL: -398.35; val LL: -398.64].....[train forward+backward+step 0.22; val forward 0.00] 
[Epoch 8/350][train LL: -397.81; val LL: -398.19].....[train forward+backward+step 0.23; val forward 0.00] 
[Epoch 9/350][train LL: -397.83; val LL: -398.25].....[train forward+backward+step 0.23; val forward 0.00] 
[Epoch 10/350][train LL: -3

In [6]:
juice.save('genetic_pc/test.jpc', pc)

In [7]:
ns = juice.load('genetic_pc/test.jpc')
pc = juice.compile(ns)
pc.to(device)

Compiling 8 TensorCircuit layers...


100%|██████████| 8/8 [00:00<00:00, 26.88it/s]


TensorCircuit(
  (input_layer_group): LayerGroup(
    (layer_0): InputLayer()
  )
  (prod_layer_0): LayerGroup(
    (layer_0): ProdLayer(
      (partitioned_nids): FastParamList(
          (0): Parameter containing: [torch.int64 of size 111 (cuda:0)]
          (1): Parameter containing: [torch.int64 of size 1 (cuda:0)]
      )
      (partitioned_cids): FastParamList(
          (0): Parameter containing: [torch.int64 of size 111x4 (cuda:0)]
          (1): Parameter containing: [torch.int64 of size 1x16 (cuda:0)]
      )
      (partitioned_u_cids): FastParamList(  (0): Parameter containing: [torch.int64 of size 258 (cuda:0)])
      (partitioned_parids): FastParamList(  (0): Parameter containing: [torch.int64 of size 258x1 (cuda:0)])
    )
  )
  (sum_layer_0): LayerGroup(
    (layer_0): SumLayer(
      (partitioned_nids): FastParamList(  (0): Parameter containing: [torch.int64 of size 112 (cuda:0)])
      (partitioned_cids): FastParamList(  (0): Parameter containing: [torch.int64 of size 